In [6]:
import gym
import random
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from collections import deque

In [7]:
class DQNAgent:
    def __init__(self,state_size,action_size,epsilon,epsilon_decay):
        self.state_size = state_size
        self.action_size = action_size
        self.batch_size = 100
        self.learning_rate = 0.01
        self.Epsilon = epsilon
        self.Gamma = 0.9
        self.Epsilon_decay = epsilon_decay
        self.Epsilon_min = 0.5
        self.memory = deque(maxlen = 10000)
        self.model = self.buildModel()
    
    def buildModel(self):
        model = Sequential()
        model.add(Dense(15,input_dim = self.state_size,activation = 'relu'))
        model.add(Dense(15,activation = 'relu'))
        model.add(Dense(self.action_size,activation = 'linear'))
        model.compile(loss = 'mse', optimizer = Adam(lr = self.learning_rate))
        return model
    
    def chooseAction(self,state):
        if (np.random.uniform() <= self.Epsilon):
            return random.randrange(self.action_size)
        action = self.model.predict(state)
        return np.argmax(action)
    
    def store(self,state,action,reward,next_state,done):
        self.memory.append((state,action,reward,next_state,done))
    
    def replay(self):
        batch = random.sample(self.memory,self.batch_size)
        cost = 0
        for state,action,reward,next_state,done in batch:
            if done:
                target  = reward
            else:
                target = reward + self.Gamma * np.amax(self.model.predict(next_state))
            current = self.model.predict(state)
            cost += abs(target - current[0][action])
            current[0][action] = target
            self.model.fit(state,current,epochs=1,verbose=0)
        if (self.Epsilon > self.Epsilon_min):
            self.Epsilon *= self.Epsilon_decay
        
    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)
        

In [8]:
np.random.seed(2)
Episodes = 2000

In [9]:
env = gym.make('Acrobot-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size,action_size,1.0,0.999)
#agent.load("Acrobot-dqn.h5")
done = False
for e in range(Episodes):
    state = env.reset()
    state = np.reshape(state, [1,state_size])
    steps = 0
    for time in range(500):
        if (e >= 1990):
            env.render()
        action = agent.chooseAction(state)
        if action != 1:
            steps +=1
        next_state,reward,done,_ = env.step(action)
        if reward == -1:
            if action == 1:
                reward = 0
            else:
                reward = -1
        else:
            reward = 100
        next_state = np.reshape(next_state, [1, state_size])
        agent.store(state,action,reward,next_state,done)
        state = next_state
        if done:
            break
    agent.replay()
    if (reward == 100):
        print("Episode: {}/{}, e: {:.2}, Time: {}, Steps: {}"
          .format(e, Episodes, agent.Epsilon,time,steps),end = '\n')            
    else:
        print("Episode: {}/{}, e: {:.2}, Time: {}, Steps: {}"
          .format(e, Episodes, agent.Epsilon,time,steps),end = '\r')                 
env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Episode: 10/2000, e: 0.99, Time: 320, Steps: 213
Episode: 54/2000, e: 0.95, Time: 488, Steps: 308
Episode: 196/2000, e: 0.82, Time: 486, Steps: 272


In [14]:
agent.save("Acrobot-dqn.h5")